In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_csv = Path("Resources/schools_complete.csv")
student_csv = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_df = pd.read_csv(school_csv)
student_df = pd.read_csv(student_csv)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(school_df, student_df, how="left", on=["school_name", "school_name"])

##  Local Government Area (LGA) Summary

In [2]:
# Total number of unique schools
total_unique_schools = len(school_data_complete['school_name'].unique())

# Total students
total_students = len(school_data_complete['Student ID'])
total_students_fmt = f"{total_students:,.0f}"

# Total budget
school_budget_df = school_data_complete[["school_name", "budget"]]

# 1. Drop duplicates: https://sparkbyexamples.com/pandas/get-unique-rows-in-pandas/
uniq_school_budget_df = school_budget_df.drop_duplicates()
uniq_school_budget_df
total_budget = uniq_school_budget_df['budget'].sum()
total_budget_fmt = f"${total_budget:,.0f}"

# Average maths score, formatted to 2 decimal places
Avg_maths_score = school_data_complete['maths_score'].mean()
Avg_maths_score_fmt = f"{Avg_maths_score:,.2f}"

# Average reading score, formatted to 2 decimal places
Avg_reading_score = school_data_complete['reading_score'].mean()
Avg_reading_score_fmt = f"{Avg_reading_score:,.2f}"

# % passing maths (the percentage of students who passed maths), formatted to 2 decimal places
pct_pass_maths = len(school_data_complete.loc[school_data_complete['maths_score'] >= 50]) / total_students * 100
pct_pass_maths_fmt = f"{pct_pass_maths:,.2f}"

# % passing reading (the percentage of students who passed reading), formatted to 2 decimal places
pct_pass_reading = (len(school_data_complete.loc[school_data_complete['reading_score'] >= 50]) / total_students * 100)
pct_pass_reading_fmt = f"{pct_pass_reading:,.2f}"

# % overall passing (the percentage of students who passed maths AND reading), formatted to 2 decimal places
pct_pass_overall = len(school_data_complete.loc[(school_data_complete['maths_score'] >= 50) & 
                                                (school_data_complete['reading_score'] >= 50)]
                      ) / total_students * 100
pct_pass_overall_fmt = f"{pct_pass_overall:,.2f}"

# Create summary output table
LGA_smy_table_data = {'Total Schools': [total_unique_schools],
                     'Total Students': [total_students_fmt],
                     'Total Budget': [total_budget_fmt],
                     'Average Maths Score': [Avg_maths_score_fmt],
                     'Average Reading Score': [Avg_reading_score_fmt],
                     '% Passing Maths': [pct_pass_maths_fmt],
                     '% Passing Reading': [pct_pass_reading_fmt],
                     '% Overall Passing': [pct_pass_overall_fmt]
                     }
area_summary = pd.DataFrame(data=LGA_smy_table_data)
display(area_summary)

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",70.34,69.98,86.08,84.43,72.81


## School Summary

In [3]:
# Total number of unique schools, school type, student population, school budget
school_type_size_budget = school_data_complete[['school_name', 'type', 'size', 'budget']]

# Create copy of original df to avoid warning of SettingWithCopyWarning
uniq_school_type_size_budget = school_type_size_budget.copy()
uniq_school_type_size_budget.drop_duplicates(inplace=True)

uniq_school_type_size_budget['Per Student Budget'] = (uniq_school_type_size_budget['budget'] / uniq_school_type_size_budget['size'])
uniq_school_type_size_budget = uniq_school_type_size_budget.set_index('school_name')

# Calculate average test scores for maths and reading
# 2. https://pandas.pydata.org/pandas-docs/version/0.25.0/reference/api/pandas.core.groupby.GroupBy.mean.html
avg_maths_score_grouped = school_data_complete.groupby('school_name')['maths_score'].mean()
avg_reading_score_grouped = school_data_complete.groupby('school_name')['reading_score'].mean()

# number of students that that pass maths, reading - grouped
# 3. https://www.statology.org/pandas-groupby-count-with-condition/
pass_maths_grouped = school_data_complete.groupby('school_name')['maths_score'].apply(lambda x: (x>=50).sum()).reset_index(name='# pass maths')
pass_reading_grouped = school_data_complete.groupby('school_name')['reading_score'].apply(lambda x: (x>=50).sum()).reset_index(name='# pass reading')

pass_maths_grouped = pass_maths_grouped.set_index('school_name')
pass_reading_grouped = pass_reading_grouped.set_index('school_name')

# number of students that that pass both maths & reading - grouped
pass_both_filter = school_data_complete.loc[(school_data_complete['maths_score'] >= 50) & (school_data_complete['reading_score']>= 50)]
pass_both_grouped = pass_both_filter.groupby('school_name').size().reset_index(name='# pass math & reading')
pass_both_grouped = pass_both_grouped.set_index('school_name')

# Merge dataframes, create a new dataframe and rename columns
# 4. https://stackoverflow.com/questions/44327999/how-to-merge-multiple-dataframes
combined_school_data1 = [uniq_school_type_size_budget, avg_maths_score_grouped, avg_reading_score_grouped, 
                         pass_maths_grouped, pass_reading_grouped, pass_both_grouped]
combined_school_data2 = pd.concat(combined_school_data1, join='outer', axis=1)
combined_school_data2
combined_school_data2.rename(columns = {'type':'School Type',
                                        'size':'Total Students',
                                        'budget':'Total School Budget',
                                        'maths_score':'Average Maths Score',
                                        'reading_score':'Average Reading Score'
                                       },
                             inplace = True)

# Get % pass maths, % pass reading, % pass overall
combined_school_data2['% Passing Maths'] = combined_school_data2['# pass maths'] / combined_school_data2['Total Students'] * 100
combined_school_data2['% Passing Reading'] = combined_school_data2['# pass reading'] / combined_school_data2['Total Students'] * 100
combined_school_data2['% Overall Passing'] = combined_school_data2['# pass math & reading'] / combined_school_data2['Total Students'] * 100

# Create per_school_summary dataframe, Drop calculation columns
# 5. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_index.html
per_school_summary = combined_school_data2.drop(['# pass maths', '# pass reading', '# pass math & reading'], axis=1)

# Drop index name
# 6. https://www.statology.org/pandas-remove-index-name/
per_school_summary.index.name = None

# sort by index
per_school_summary.sort_index(inplace=True)

# Display dataframe with formatted columns and no spillover columns
# Want to keep original dataframe for further calculations
# 7. https://stackoverflow.com/questions/26873127/show-dataframe-as-table-in-ipython-notebook
# 8. https://www.geeksforgeeks.org/formatting-integer-column-of-dataframe-in-pandas/
per_school_summary_fmt = per_school_summary.copy()
per_school_summary_fmt['Total School Budget'] = per_school_summary_fmt['Total School Budget'].apply(lambda x: '${:,.2f}'.format(x))
per_school_summary_fmt['Per Student Budget'] = per_school_summary_fmt['Per Student Budget'].apply(lambda x: '${:,.2f}'.format(x))

display(per_school_summary_fmt)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
Pena High School,Independent,962,"$585,858.00",$609.00,72.088358,71.613306,91.683992,86.590437,79.209979


## Top Performing Schools (By % Overall Passing)

In [4]:
# Sort the schools by % Overall Passing in descending order and display the top 5 rows
# 9. https://stackoverflow.com/questions/49632059/pandas-sort-values-to-get-top-5-for-each-column-in-a-groupby
top_schools = per_school_summary_fmt.sort_values(['% Overall Passing'], ascending=[False]).head(5)

display(top_schools)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,90.797699,87.396849,79.419855


## Bottom Performing Schools (By % Overall Passing)

In [5]:
# Sort the schools by % Overall Passing in asscending order and display the top 5 rows
bottom_schools = per_school_summary_fmt.sort_values(['% Overall Passing'], ascending=[True]).head(5)

display(bottom_schools)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880


## Maths Scores by Year

In [6]:
# Create a series for each year & dynamically create variable name and column heading
# 10. https://plainenglish.io/blog/how-to-dynamically-declare-variables-inside-a-loop-in-python

for i in range(9, 13):
    globals()[f'Series_math_{i}'] = school_data_complete.loc[(school_data_complete['year'] == i)]
    globals()[f'Group_Maths_Yr_{i}'] = globals()[f'Series_math_{i}'].groupby('school_name')['maths_score'].mean().reset_index(name='Year '+str(i))
    globals()[f'Group_Maths_Yr_{i}'] = globals()[f'Group_Maths_Yr_{i}'].set_index('school_name')

# Merge series, create a new dataframe
maths_scores_by_year = [Group_Maths_Yr_9, Group_Maths_Yr_10, Group_Maths_Yr_11, Group_Maths_Yr_12]
maths_scores_by_year = pd.concat(maths_scores_by_year, join='outer', axis=1)

# Remove index name, sort by index and display
maths_scores_by_year.index.name = None
maths_scores_by_year.sort_index(inplace=True)

display(maths_scores_by_year)

,Year 9,Year 10,Year 11,Year 12
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393
Pena High School,71.996364,72.396000,72.523438,71.187845


## Reading Scores by Year 

In [7]:
# Create a series for each year & dynamically create variable name and column heading
for i in range(9, 13):
    globals()[f'Series_read_{i}'] = school_data_complete.loc[(school_data_complete['year'] == i)]
    globals()[f'Group_Read_Yr_{i}'] = globals()[f'Series_read_{i}'].groupby('school_name')['reading_score'].mean().reset_index(name='Year '+str(i))
    globals()[f'Group_Read_Yr_{i}'] = globals()[f'Group_Read_Yr_{i}'].set_index('school_name')

# Merge series, create a new dataframe
reading_scores_by_year = [Group_Read_Yr_9, Group_Read_Yr_10, Group_Read_Yr_11, Group_Read_Yr_12]
reading_scores_by_year = pd.concat(reading_scores_by_year, join='outer', axis=1)

# Remove index name, sort by index and display
reading_scores_by_year.index.name = None
reading_scores_by_year.sort_index(inplace=True)

display(reading_scores_by_year)

,Year 9,Year 10,Year 11,Year 12
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521
Pena High School,70.949091,72.324000,71.703125,71.513812


## Scores by School Spending

In [8]:
# Create bins for spending and labels
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Slice per_school_summary df via Per student budget, set bins and labels & create new df
school_spending_ranges = pd.cut(per_school_summary['Per Student Budget'],bins=spending_bins, labels=labels)
school_spending_ranges = pd.DataFrame(school_spending_ranges)

school_spending_ranges = school_spending_ranges.rename(columns = {'Per Student Budget':'Spending Ranges (Per Student)'})

# Merge the two dfs
school_spending_df = pd.merge(per_school_summary, school_spending_ranges, how='outer', left_index=True, right_index=True)

# Calculate the mean for the following columns, grouped by Spending Ranges (Per Student)
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Maths Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Maths"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

# Merge series, create a new dataframe, sort by index
spending_summary_data = [spending_math_scores, spending_reading_scores, 
                         spending_passing_math, spending_passing_reading, overall_passing_spending]

spending_summary = pd.concat(spending_summary_data, join='outer', axis=1)

spending_summary.sort_index(inplace=True)

# Display values with 2 decimal places just for this table
# 11. https://pandas.pydata.org/docs/user_guide/style.html
spending_summary.style.format(precision=2)

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.36,70.72,88.84,86.39,76.72
$585-630,72.07,71.03,91.52,87.29,79.88
$630-645,69.85,69.84,84.69,83.76,71.00
$645-680,68.88,69.05,81.57,81.77,66.76


## Scores by School Size

In [9]:
# Create bins for school size and labels
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Slice per_school_summary df via Total Students, set bins and labels & create new df
total_student_ranges = pd.cut(per_school_summary['Total Students'],bins=size_bins, labels=labels)
total_student_ranges = pd.DataFrame(total_student_ranges)

total_student_ranges = total_student_ranges.rename(columns = {'Total Students':'School Size'})

# Merge the two dfs
total_student_df = pd.merge(per_school_summary, total_student_ranges, how='outer', left_index=True, right_index=True)

# Calculate the mean for the following columns, grouped by School Size
schoolsize_math_scores = total_student_df.groupby(["School Size"])["Average Maths Score"].mean()
schoolsize_reading_scores = total_student_df.groupby(["School Size"])["Average Reading Score"].mean()
schoolsize_passing_math = total_student_df.groupby(["School Size"])["% Passing Maths"].mean()
schoolsize_passing_reading = total_student_df.groupby(["School Size"])["% Passing Reading"].mean()
overall_passing_schoolsize = total_student_df.groupby(["School Size"])["% Overall Passing"].mean()

# Merge series, create a new dataframe, sort by index
size_summary_data = [schoolsize_math_scores, schoolsize_reading_scores, 
                     schoolsize_passing_math, schoolsize_passing_reading, overall_passing_schoolsize]

size_summary = pd.concat(size_summary_data, join='outer', axis=1)

size_summary.sort_index(inplace=True)

display(size_summary)

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


## Scores by School Type

In [10]:
# Calculate the mean for the following columns, grouped by School Type
schooltype_math_scores = per_school_summary.groupby(['School Type'])["Average Maths Score"].mean()
schooltype_reading_scores = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
schooltype_passing_math = per_school_summary.groupby(["School Type"])["% Passing Maths"].mean()
schooltype_passing_reading = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
overall_passing_schooltype = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

# Merge series, create a new dataframe, sort by index
type_summary_data = [schooltype_math_scores, schooltype_reading_scores, 
                     schooltype_passing_math, schooltype_passing_reading, overall_passing_schooltype]
type_summary = pd.concat(type_summary_data, join='outer', axis=1)

type_summary.sort_index(inplace=True)

display(type_summary)

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
